# 動的計画法

## 動的計画法とは：問題にチャレンジする前に


問題：[ALDS1_5_A](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/5/ALDS1_5_A) (Exhaustive Search)

In [ ]:
%%writefile input.dat
5
1 5 7 10 21
4
2 4 17 8

### 以前の方法

In [ ]:
%%writefile test.py
def solve(i, m):
  if m == 0: return True
  if i >= n: return False
  tmp = solve(i + 1, m) or solve(i + 1, m - A[i])
  return tmp

n = int(input())
A = list(map(int, input().split()))
q = int(input())
m = list(map(int, input().split()))

for x in m: print('yes' if solve(0, x) else 'no')

In [ ]:
!python3 test.py < input.dat

### メモ化


同じ計算を何度もする無駄を省くために，solveの結果を記憶することにする（**メモ化**，memoization）。

#### ライブラリを使う


定義の前に次のコードを追加すると，その関数がメモ化される。

In [3]:
%%writefile test.py
from functools import cache

@cache

def solve(i, m):
  if m == 0: return True
  if i >= n: return False
  tmp = solve(i + 1, m) or solve(i + 1, m - A[i])
  return tmp

n = int(input())
A = list(map(int, input().split()))
q = int(input())
m = list(map(int, input().split()))

for x in m: print('yes' if solve(0, x) else 'no')

Overwriting test.py


In [2]:
!python3 test.py < input.dat

/bin/bash: line 1: input.dat: No such file or directory


#### 自分で実装する

In [4]:
%%writefile test.py
dp = {} # 変更点1：結果を記録するための辞書

def solve(i, m):
  # 変更点2：辞書に入っているならそれを返す。
  if (i, m)in dp: return dp[(i, m)]
  if m == 0: return True
  if i >= n: return False
  tmp = solve(i + 1, m) or solve(i + 1, m - A[i])
  # 変更点3：辞書に入れる。
  dp[(i, m)] = tmp
  return tmp

n = int(input())
A = list(map(int, input().split()))
q = int(input())
m = list(map(int, input().split()))

for x in m: print('yes' if solve(0, x) else 'no')

Overwriting test.py


In [ ]:
!python3 test.py < input.dat

## フィボナッチ数列


問題：[ALDS1_10_A](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_A) (Fibonacci Numbers)

<img src="https://ndlsearch.ndl.go.jp/thumbnail/9784794964540.jpg" width="150"/> エンツェンスベルガー『数の悪魔』（晶文社, 2000）

フィボナッチ数列の定義には二つの流儀がある。

-   定義1: $f_n = f_{n-1} + f_{n-2},\quad f_0 = 1,\quad f_1 = 1.$
-   定義2: $F_n = F_{n-1} + F_{n-2},\quad F_0 = 0,\quad F_1 = 1.$

教科書で採用しているのは定義1だが，定義2の方が良い。$f_n=F_{n+1}$ で相互に変換できるから，どちらでもよいと思うかもしれないが，フィボナッチ数列に関する有名な定理で，定義2でしか成り立たないものがある。例えば，$\mathrm{gcd}(F_m, F_n)=F_{\mathrm{gcd}(m, n)}$ は，定義2でしか成り立たない（$m=3, n=6$ として試してみよ）。

そこでこの資料では，定義2の$F_n$ を求める方法を実装して，$f_n=F_{n+1}$ から $f_n$ を求めることにする。

フィボナッチ数列に関する話題。★はAizu Online Judgeで使えるもの（NumPyやSymPyを使うコードは使えない）。

-   再帰
    -   単純な実装★
    -   メモ化
        -   ライブラリ★
        -   自分で書く★
-   ループ★
-   ♠漸化式を解く
    -   数式処理
    -   紙とペン（**高校数学**）★
-   ♠行列の累乗
    -   NumPy
    -   対角化（**線形代数**）
        -   数式処理
        -   紙とペン★
-   ♠母関数（**微分積分**）
    -   テイラー近似式
    -   整数計算★
-   ♠アート

$f_{10}=F_{11}=89$ を確認しながら進める。

### 再帰

#### 単純な実装★

In [ ]:
%%writefile test.py
def fibonacci(n):
  # n <= 1 なら n を返す。
  # 再帰的に計算した結果を返す。

n = int(input())
print(fibonacci(n + 1)) # f(n) = F(n + 1)

In [ ]:
!echo 10 | python3 test.py

うまく行ったら提出してみる。

#### メモ化

##### ライブラリ★

In [ ]:
%%writefile test.py
from functools import cache

@cache
def fibonacci(n):
  if n <= 1: return fibonacci(n - 1) + fibonacci

In [ ]:
!echo 10 | python3 test.py

うまく行ったら提出してみる。

##### 自分で書く★


一度計算した結果を辞書に記録する。（フィボナッチ数列の場合は，番号が0, 1, 2, … のように等間隔だから，リストを使ってもよい。）

In [5]:
%%writefile test.py
F = {} # 結果を記録するための辞書

def fibonacci(n):
  if n in F: return F[n] # 辞書にあるならそれを返す。
  if n <= 1: return n # 辞書に無いなら
  tmp = fibonacci(n - 2) + fibonacci(n - 1) # 計算して
  F[n] = tmp # 辞書に入れて
  return tmp # 返す。

n = int(input())
print(fibonacci(n + 1))

Overwriting test.py


In [ ]:
!echo 10 | python3 test.py

### ループ

In [ ]:
%%writefile test.py
def fibonacci(n):
  a, b = 0, 1
  for i in range(n - 1): a, b = b, a + b # 必要な分だけ更新を繰り返す。
  return b

n = int(input())
print(fibonacci(n + 1))

In [6]:
!echo 10 | python3 test.py

89


### ♠漸化式を解く


次の三項間漸化式を解く。
$$F_n = F_{n-1} + F_{n-2},\quad F_0 = 0,\quad F_1 = 1.$$

#### 数式処理


数式処理で正確に解く（[WolframAlpha](https://www.wolframalpha.com/input?i=F%28n%29%3DF%28n-1%29%2BF%28n-2%29%2CF%280%29%3D0%2CF%281%29%3D1&lang=ja)）。

Python（SymPy）なら，`rsolve`に`左辺 = 0`あるいは`Eq(左辺, 右辺)`の形式で漸化式を与える。

In [ ]:
from sympy import *

F = Function('F')
var('n')
result = rsolve(F(n) - F(n-1) - F(n-2), F(n), {F(0): 0, F(1): 1})
#result = rsolve(Eq(F(n), F(n-1) + F(n-2)), F(n), {F(0): 0, F(1): 1})

`n`に $0$ から $11$ までの数を代入して，正しいことを確かめる。

In [ ]:
[simplify(result.subs(n, i)) for i in range(12)]

#### 紙とペン（**高校数学**）


漸化式の特性方程式は
$$x^2-x-1=0$$
で，解は $\dfrac{1\pm\sqrt{5}}{2}$ である。特性方程式が異なる解 $\alpha, \beta$ をもつとき，漸化式の一般項は $A, B$ を定数として，
$$F_n=A\alpha^n+B\beta^n$$
と書ける（**高校数学の参考書等を参照**）。特性方程式の解を $\alpha:=\dfrac{1+\sqrt{5}}{2}$，$\beta:=\dfrac{1-\sqrt{5}}{2}$ として，$F_0=0, F_1=1$ となるように $A, B$ を定めると，
$$F_n=\dfrac{1}{\sqrt 5}(\alpha^n-\beta^n)$$
となる（連立方程式 $A+B=0, A\alpha+B\beta=1$ の解は $A=1/\sqrt{5}, B=-1/\sqrt{5}$）。

この結果を使って正確に計算する。

In [ ]:
from sympy import *

def fibonacci(n):
  alpha = (1 + sqrt(5)) / 2
  beta  = (1 - sqrt(5)) / 2
  return simplify((alpha**n - beta**n) / sqrt(5))

print(fibonacci(10 + 1))

問題の制約は $n\le 44$ で，このくらいなら数値計算（近似計算）でも正しい結果が得られる。♠問われているのが $f_{44}=F_{45}$ までなのは，32ビット整数の最大値 $2^{31}-1=2147483647$ を意識してのことだろう。$F_{47}=2971215073$（$=f_{46}$） が32ビット整数の範囲を超える。これを超えてもPythonなら問題なく計算できるが，C言語やC++では注意が必要になる。

次のコードなら，Aizu Online Judgeで提出できる。

In [ ]:
%%writefile test.py
def fibonacci(n):
  alpha = (1 + 5**0.5) / 2
  beta  = (1 - 5**0.5) / 2
  return round((alpha**n - beta**n) / 5**0.5)

n = int(input())
print(fibonacci(n + 1))

In [ ]:
!echo 10 | python3 test.py

♠数値計算で最初にうまく行かなくなるのは $f_{70}=F_{71}=308061521170129$ である。

In [ ]:
# 浮動小数点数による近似計算
def f1(n):
  alpha = (1 + 5**0.5) / 2
  beta  = (1 - 5**0.5) / 2
  return round((alpha**n - beta**n) / 5**0.5) # roundで整数にする。

from sympy import sqrt, simplify

# 数式処理による厳密計算
def f2(n):
  alpha = (1 + sqrt(5)) / 2
  beta  = (1 - sqrt(5)) / 2
  return simplify((alpha**n - beta**n) / sqrt(5))

n = 0
while True:
  v1, v2 = f1(n + 1), f2(n + 1)
  if v1 != v2:
    print(f'{n = }')
    print(f'近似計算：{v1}')
    print(f'厳密計算：{v2}')
    break
  n += 1

♠問題：[049 - Fibonacci Easy (mod 1000000007)](https://atcoder.jp/contests/math-and-algorithm/tasks/math_and_algorithm_ap)

これまでに学んだ手法から，適切なものを選べば解ける。

In [ ]:
%%writefile test.py
#自分で書く

In [ ]:
!echo 8691200 | python3 test.py

### ♠行列の累乗


フィボナッチ数列の漸化式は行列を使って
$$\begin{pmatrix}F_{n+1}&F_n\\F_{n}&F_{n-1}\end{pmatrix}
=\begin{pmatrix}F_n+F_{n-1}&F_{n-1}+F_{n-2}\\F_n&F_{n-1}\end{pmatrix}
=\begin{pmatrix}1&1\\1&0\end{pmatrix}\begin{pmatrix}F_n&F_{n-1}\\F_{n-1}&F_{n-2}\end{pmatrix}
=\begin{pmatrix}1&1\\1&0\end{pmatrix}^{n-1}\begin{pmatrix}F_2&F_1\\F_1&F_0\end{pmatrix}=\begin{pmatrix}1&1\\1&0\end{pmatrix}^n$$
と表せる（$F_0=0, F_1=1, F_2=1$ に注意）。

$A=\begin{pmatrix}1&1\\1&0\end{pmatrix}$とすると，$F_n$ は $A^n$ の $(1, 2)$ 成分が $F_n$ である。例えば，
$$\begin{pmatrix}F_3&F_2\\F_2&F_1\end{pmatrix}
=\begin{pmatrix}1&1\\1&0\end{pmatrix}^2
=\begin{pmatrix}2&1\\1&1\end{pmatrix}$$
だから，$F_2=1$ である。

$F_{11}=f_{10}$ が欲しければ，$A^{11}$ の $(1, 2)$ 成分，あるいは，$A^{10}$ の $(1, 1)$ 成分を求めればよい。$A^{10}$ を直接計算するのは大変だから，[WolframAlphaを使う](https://www.wolframalpha.com/input?i=%7B%7B1%2C1%7D%2C%7B1%2C0%7D%7D%5E10&lang=ja)。

Python（NumPy）では，行列の累乗は`np.linalg.matrix_power(A, n)`で計算する。`A**n`ではないことに注意。$(1, 2)$ 成分には`[0, 1]`でアクセスする。

In [ ]:
import numpy as np

A = np.array([[1, 1], [1, 0]])
n = 2
B = np.linalg.matrix_power(A, n)
print(B)
print(B[0, 1]) # B=A^nの(1, 2)成分

n = 10 + 1
B = np.linalg.matrix_power(A, n)
print(B)
print(B[0, 1])

関数にすると次のとおり。

In [ ]:
import numpy as np

A = np.array([[1, 1], [1, 0]])

def fibonacci(n):
  return (np.linalg.matrix_power(A, n)[0, 1])

print(fibonacci(10 + 1))

#### ♠大きな数


♠問題：[054 - Fibonacci Hard (mod 1000000000)](https://atcoder.jp/contests/math-and-algorithm/tasks/math_and_algorithm_at)）。

NumPyの整数のデフォルトは64ビットだから，それを超える計算はできない。できたとしても，$F_{876543210987654321}$ のような大きな数の計算には時間がかかるだろう。次のような方法が考えられる（AtCoderではNumPyが使えるから，行列の乗算の実装の必要はない）。

> Pythonの桁数無限の整数をラップして，乗算と加算をmod 10^9で行うオブジェクトを作る。それを要素とするNumPyの行列\[\[1,1\],\[1,0\]\]の876543210987654321乗の(1,2)成分を求めるコード

#### 対角化（**線形代数**）


行列の累乗は，対角化を使って計算する。

$P=\begin{pmatrix}\beta&\alpha\\1&1\end{pmatrix}$, $D=\begin{pmatrix}\beta&0\\0&\alpha\end{pmatrix}$とすると，$P^{-1}=\dfrac{1}{\sqrt 5}\begin{pmatrix}-1&\alpha\\1&-\beta\end{pmatrix}$ で，$A$ は
$$P^{-1}AP=D$$
と対角化される。

$A=PDP^{-1}$ だから，
$$A^n=(PDP^{-1})^n=(PDP^{-1})\cdots(PDP^{-1})=PD^nP^{-1}=\dfrac{1}{\sqrt 5}\begin{pmatrix}\alpha^{n+1}-\beta^{n+1}&\alpha^n-\beta^n\\\alpha^n-\beta^n&\alpha^{n-1}-\beta^{n-1}\end{pmatrix}$$
となる。この $(1, 2)$ 成分が $F_n$ だから，
$$F_n=\dfrac{1}{\sqrt 5}(\alpha^{n}-\beta^{n}).$$

##### 数式処理


[WolframAlphaで対角化](https://www.wolframalpha.com/input?i=diagonalize+%7B%7B1%2C1%7D%2C%7B1%2C0%7D%7D&lang=ja)

Python（SymPy）なら，`diagonalize`（対角化）を使う。

In [ ]:
from sympy import *

def fibonacci(x):
  var('n')
  A = Matrix([[1, 1], [1, 0]])
  P, D = A.diagonalize()
  A_n = P * D**n * P.inv()
  return simplify(A_n[0, 1].subs(n, x)) # (1,2)成分のnを引数で置き換える。

print(fibonacci(10 + 1))

##### 紙とペン


**長崎憲一,
横山利幸『明解 線形代数』（培風館, 改訂版, 2024）** の方法で対角化する。その結果として，漸化式の解と同じものを得て，漸化式の解を使ったコードと同じコードを書くことになる。♠$A$ は対称行列だから直交行列で対角化できるのだが，ここではそこまでしなくてよい（同書§23を参照）。

### ♠母関数（**微分積分**）


<img src="https://ndlsearch.ndl.go.jp/thumbnail/9784320124646.jpg" width="150"/> Graham, Knuth, Patashnik『コンピュータの数学』（共立出版, 第2版, 2020） 原題は『Concrete Mathematics』。連続（CONtinuous）と離散（disCRETE）のブレンドが大事だということである。

> この本全体で最も重要な概念，すなわち，**母関数**（generating function）の概念に到達した。

数列 $\langle a_0, a_1, a_2, \dots\rangle$に対して，
$$A(z):=\sum_{k\ge 0}a_kz^k=a_0+a_1z+a_2z^2+\dots$$
つまり，$z^k$ の係数が $a_k$ であるような $z$ の多項式（で表される関数）を，その数列の母関数という。

#### 母関数の例


$\langle 1, 2, 3, 4,\dots\rangle$ （一般項は $a_n=n+1$）の母関数は $A(z)=\dfrac{1}{(1-z)^2}$である。

1.  数列から母関数へ
    1.  定義に基づいて計算すると，$\displaystyle\sum_{k\ge 0}(k+1)z^k=\dfrac{1}{(1-z)^2}$ である（[WolframAlpha](https://www.wolframalpha.com/input?i=Sum+%28k%2B1%29z%5Ek%2C+k%3D0+to+infinity&lang=ja)）。
    2.  母関数を求める機能を使う（[WolframAlpha](https://www.wolframalpha.com/input?i=GeneratingFunction%5B1%2Bk%2Ck%2Cz%5D&lang=ja)）。
2.  母関数から数列へ
    1.  母関数 $A$ の，$z=0$ におけるテイラー近似式の $z^n$ の係数が $a_n$ である。つまり，$a_n=\dfrac{A^{(n)}(0)} {n!}$ で，この場合は $n+1$ になるはず（[WolframAlpha](https://www.wolframalpha.com/input?i=SeriesCoefficient%5B1%2F%281-z%29%5E2%2C%7Bz%2C0%2Cn%7D%5D&lang=ja)）。
    2.  テイラー近似式を求める機能を使う（[WolframAlpha](https://www.wolframalpha.com/input?i=series+1%2F%281-z%29%5E2+at+z%3D0+to+order+10&lang=ja)）。

テイラー近似式については，**長崎憲一ほか『明解 微分積分』（培風館, 改訂版, 2019）** の§9を参照。

Python（SymPy）で1の1，つまり $\displaystyle\sum_{k\ge 0}(k+1)z^k=\dfrac{1}{(1-z)^2}$を確認する。

In [ ]:
from sympy import *

var('k z')
simplify(Sum((k + 1) * z**k, (k, 0, oo)).doit())

2の2，つまりテイラー近似式を確認する。

In [ ]:
A = 1 / ((1 - z)**2)
A.series(z, 0, 10)

#### 母関数の応用例


母関数には，今後さまざまなところで再会するだろう。

-   統計学では，確率分布の性質を調べるのに母関数が使われる。
-   離散数学では，漸化式や組合せを調べるのに使われる。

例：1円，5円，10円，50円，100円硬貨を使って20円にする方法は何通りあるか。490円にする方法は？（使わない硬貨があってもよい。）

母関数は次のとおり（前掲の『コンピュータの数学』を参照）。

In [ ]:
from sympy import *
A = 1
for k in [1, 5, 10, 50, 100]: A /= 1 - z**k
A

$z=0$ におけるテイラー近似式の $z^{20}$ の係数が，20円にする方法の数である。

In [ ]:
n = 20
print(A.series(z, 0, n + 1).coeff(z**n))

490円にする方法：[WolframAlpha](https://www.wolframalpha.com/input?i=SeriesCoefficient%5B1%2F%28%281-z%29%281-z%5E5%29%281-z%5E10%29%281-z%5E50%29%281-z%5E100%29%29%2C%7Bz%2C0%2C490%7D%5D&lang=ja)

ただし，この方法を数式処理が無い環境で使うのは大変だし，数式処理があっても遅いと使えないという問題がある。そういう場合は，教科書17.1節のように，動的計画法を使うことになる。有名な問題だから，いろんな文献で紹介されている。例：https://github.com/okumuralab/algo-c/blob/main/src/change.c （奥村晴彦『C言語による標準アルゴリズム事典』（技術評論社, 改訂新版, 2018））のコードを実行してみる。

In [ ]:
%%bash
wget -nc -q https://raw.githubusercontent.com/okumuralab/algo-c/refs/heads/main/src/change.c
gcc change.c
./a.out | head
./a.out | tail

#### フィボナッチ数列の母関数


フィボナッチ数列の母関数は $A(z)=\dfrac{z}{1-z-z^2}$ である。理由：フィボナッチ数列 $\langle0, 1, 1, 2,\dots\rangle$ の母関数を $A(z)$ とすると，
$$(1-z-z^2)A(z)=A(z)-zA(z)-z^2A(z)=(F_0+F_1z+F_2z^2+\cdots)-(F_0z+F_1z^2+F_2z^2+\cdots)-(F_0z^2+F_1z^3+F_2z^4+\cdots)=F_0+(F_1-F_0)z+(F_2-F_1-F_0)z^2+\cdots=z.$$

1.  数列から母関数へ
    1.  定義に基づいて計算すると，$\displaystyle\sum_{k\ge 0}F_kz^k=\dfrac{z}{1-z-z^2}$ である（[WolframAlpha](https://www.wolframalpha.com/input?i=Sum+Fibonacci%28k%29z%5Ek%2C+k%3D0+to+infinity&lang=ja)）。
    2.  母関数を求める機能を使う（[WolframAlpha](https://www.wolframalpha.com/input?i=GeneratingFunction%5BFibonacci%5Bk%5D%2Ck%2Cz%5D&lang=ja)）。
2.  母関数から数列へ
    1.  母関数 $A$ の，$z=0$ におけるテイラー近似式の $z^n$ の係数が $a_n$ である。つまり，$a_n=\dfrac{A^{(n)}(0)} {n!}$（[WolframAlpha](https://www.wolframalpha.com/input?i=SeriesCoefficient%5Bz%2F%281-z-z%5E2%29%2C%7Bz%2C0%2Cn%7D%5D&lang=ja)）。
    2.  テイラー近似式を求める機能を使う（[WolframAlpha](https://www.wolframalpha.com/input?i=series+z%2F%281-z-z%5E2%29+at+z%3D0+to+order+10&lang=ja)）。

In [ ]:
from sympy import *

def fibonacci(n):
  var('z')
  A = z / (1 - z - z**2)
  return diff(A, z, n).subs(z, 0) / factorial(n)
  #return A.series(z, 0, n + 1).coeff(z**n)

print(fibonacci(10 + 1))

#### 整数計算


数式処理（PyhtonならSymPy）があれば，母関数のテイラー近似式からフィボナッチ数列を求められる。しかし，Aizu Online Judgeのように，SymPyが使えない環境もある。ここでは，そういう環境でも使える，母関数の活用法を紹介する。

試しに，$A(\frac{1}{100})$ を求めてみよう。

In [ ]:
from sympy import *

var('z')
A = Lambda(z, z / (1 - z - z**2))
N(A(Rational(1, 100)))

2桁ずつ読んでいくと，0, 01, 01, 02, 03, 05, 08, 13, 21，とフィボナッチ数列になっている。このことは，$A(z)=F_0+F_1z+F_2z^2+\dots$ だから，$A(\frac{1}{100})$ が
$$F_0+\dfrac{F_1}{100}+\dfrac{F_2}{10000}+\dfrac{F_3}{1000000}+\dots
=0+\dfrac{1}{100}+\dfrac{1}{10000}+\dfrac{2}{1000000}+\dots
=0.010102\cdots$$
となることから納得できる。

例えば $F_3$ を知りたいときは，

1.  $A(\frac{1}{100})$ に$1000000=100^3$を掛けて，$10102.0\dots$ として
2.  整数部分だけ残して，$10102$ として，
3.  $100$ で割った余りを求めて，$2$ を得る

とすればよさそうである。

しかし，試してみると，途中でうまく行かなくなる（$55$ の次は $89$ のはず）。

In [ ]:
def fibonacci(n):
  var('z')
  A = Lambda(z, z / (1 - z - z**2))
  p = 100
  return int(p**n * A(Rational(1, p))) % p

[fibonacci(n) for n in range(15)]

うまく行かなくなるのは，$89$ の次が $144$（3桁）で，2桁ずつ読む方法では扱えないからである。

対策として，$A(\frac{1}{100})$ ではなく，$A(\frac{1}{10^n})$ として，各数を表すのに使う桁数を増やしていく。そのために`100`を`10**n`に変える（このパラメータを $p$ とする）。実は $p$ はもっと小さくてもよい（例：$p=2^n+1$，あるいは，$p=2^{n+1}$）。

In [ ]:
def fibonacci(n):
  var('z')
  A = Lambda(z, z / (1 - z - z**2))
  p = 10**n
  return int(p**n * A(Rational(1, p))) % p

[fibonacci(n) for n in range(15)]

$A(\frac{1}{p})=\dfrac{\frac{1}{p}}{1-p-\frac{1}{p^2}}$ の分母と分子に $p^2$ を掛けて $A(\frac{1}{p})=\dfrac{p}{p^2-p-1}$ とすると，計算しやすくなる。

In [ ]:
def fibonacci(n):
  p = 10**n
  return (p**n * p // (p**2 - p - 1)) % p

[fibonacci(n) for n in range(15)]

これでオンラインジャッジも通るはずだが，もう少し工夫できる。まず，$F_n$ を求めるのに $10^n$ として 10進数で $n$ 桁使うのは無駄である。$2^{n+1}$ 程度でよい。

In [ ]:
def fibonacci(n):
  p = 2**(n + 1)
  return (p**n * p // (p**2 - p - 1)) % p

[fibonacci(n) for n in range(15)]

さらに，累乗「`2**(n + 1)`」の代わりにビットシフト「`2 << n`」，剰余「`% p`」の代わりにビット論理積「`& (p - 1)`」を使って高速化を試みる。

In [ ]:
%%writefile test.py
def fibonacci(n):
  p = 2 << n
  return ((p << (n * n + n)) // ((p << (n + 1)) - p - 1)) & (p - 1)

n = int(input())
print(fibonacci(n + 1))

In [ ]:
!echo 10 | python3 test.py

### ♠アート


フィボナッチ数列はアートにも使われる。特に，フィボナッチ数列の隣接2項の比 $\dfrac{F_n}{F_{n-1}}$ は，$n$ が大きくなると**黄金比** $\phi:=\dfrac{1+\sqrt 5}{2}$ に近づく。

[フィボナッチ数（Wikipedia）](https://ja.wikipedia.org/wiki/%E3%83%95%E3%82%A3%E3%83%9C%E3%83%8A%E3%83%83%E3%83%81%E6%95%B0)

## 最長共通部分列

### 準備：二次元配列の初期化方法

In [ ]:
# 悪い例
A = [[0] * 3] * 4
A

実体が同じリストを4個作ってしまう。一部を変えると全部変わる。

In [ ]:
A[0][1] = 1
A

In [ ]:
# 良い例
A = [[0] * 3 for _ in range(4)]
A

In [ ]:
A[0][1] = 1
A

### 本題


問題：[ALDS1_10_C](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_C) (Longest Common Subsequence)

In [ ]:
%%writefile input.dat
3
abcbdab
bdcaba
abc
abc
abc
bc

教科書に合わせて，1オリジンのコードを書く。

In [7]:
%%writefile test.py
# 0オリジンでもできる
def lcs(X, Y):
  m, n = len(X), len(Y)
  c = [[0] * (n + 1) for _ in range(m + 1)] # 文字列Xを1オリジンにする（先頭は使わない）
  # 文字列Yを1オリジンにする（先頭は使わない）
  # c[i][j]の初期化。C[i][j]はX[1..i]とY[1..j]のLCSの長さ

  for i in range(m): # Xの文字についてのループ
    for j in range(n): # Yの文字についてのループ
      if X[i] == Y[j]: c[i + 1][j + 1] = c[i][j] + 1 # 自分で書く
      else: c[i + j][j + 1] = max(c[i][j + 1], c[i + 1][j])
  return c[m][n]

q = int(input())
for _ in range(q):
  s1 = input()
  s2 = input()
  print(lcs(s1, s2))

Overwriting test.py


In [ ]:
!python3 test.py < input.dat

TLEになる場合はPyPyを使う。あるいはC++に翻訳する。

♠[F - LCS](https://atcoder.jp/contests/dp/tasks/dp_f)

## 連鎖行列積


問題：[ALDS1_10_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_B) (Matrix Chain Multiplication)

In [ ]:
%%writefile input.dat
6
30 35
35 15
15 5
5 10
10 20
20 25

In [ ]:
%%writefile test.py
n = int(input())
p = [None] * (n + 1)
m = [[0] * (n + 1) for _ in range(n + 1)]

for i in range(1, n + 1):
  p[i - 1], p[i] = map(int, input().split())

for l in range(2, n + 1):
  for i in range(1, n - l + 2):
    j = i + l - 1
    m[i][j] = min(m[i][j], m[i][k] + m[k + 1][j] + p[i -1] * p[k] * p[j]):

print(m[1][n])

In [ ]:
!python3 test.py < input.dat

♠行列の積はNumPyで計算するのが一般的である。`M1`, `M2`, `M3`の積を計算するときに，ここで扱ったようなことは考慮されるか？

## 宿題


以下の問題をAC（Accepted）にする。Pythonを使うこと。

-   [ALDS1_10_A](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_A) (Fibonacci Numbers)
-   ♠[049 - Fibonacci Easy (mod 1000000007)](https://atcoder.jp/contests/math-and-algorithm/tasks/math_and_algorithm_ap)
-   ♠[054 - Fibonacci Hard (mod 1000000000)](https://atcoder.jp/contests/math-and-algorithm/tasks/math_and_algorithm_at)）
-   [ALDS1_10_C](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_C) (Longest Common Subsequence)
-   [ALDS1_10_B](https://onlinejudge.u-aizu.ac.jp/courses/lesson/1/ALDS1/10/ALDS1_10_B) (Matrix Chain Multiplication)

以上